# Predictor

In [1]:
#| default_exp predictor

In [2]:
#| export 
import torch
from CodeCheckList.tokenizer import CodeTokenizer
from transformers import AutoModelForMaskedLM, BatchEncoding

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
class Predictor:
    """Predictor Module"""
    def __init__(self, tokenizer, model, gpu_available):
        self.model = model
        self.tokenizer = tokenizer
        self.gpu_available = gpu_available

    def __call__(self, masked_code_encoding: BatchEncoding, code_encoding: BatchEncoding, top_k: int):
        masked_indexes = list(map(lambda entry: entry[0],
            list(filter(lambda entry: True if entry[1] == self.tokenizer.tokenizer.mask_token_id else False, enumerate(masked_code_encoding['input_ids'])))))
        code_encoding['input_ids'][0] = torch.tensor([torch.tensor(input_id) for input_id in masked_code_encoding['input_ids']])
        tokens_tensor = code_encoding['input_ids']
        attention_mask = code_encoding['attention_mask']
        if(self.gpu_available):
            tokens_tensor = tokens_tensor.to('cuda:3')
            attention_mask = attention_mask.to('cuda:3')
        model_input = {'input_ids' : tokens_tensor,
          'attention_mask' : attention_mask}
        model_prediction = self.model(**model_input)
        preditions = []
        for k_index in range(0, top_k):
            preditions.append(code_encoding['input_ids'][0].tolist().copy())
        for masked_index in masked_indexes:
            values, predictions = model_prediction['logits'][0][masked_index].topk(top_k)
            for k_index in range(0, top_k):
                preditions[k_index][masked_index] = predictions[k_index]
        prediction_list = list(map(lambda prediction: self.tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == self.tokenizer.tokenizer.bos_token_id or 
                    token_id == self.tokenizer.tokenizer.eos_token_id else True, prediction))), preditions))
        return prediction_list

    @staticmethod
    def from_pretrained(
        name_or_path: str,          #name or path of the model
        tokenizer: CodeTokenizer,   #tokenizer, has to be of the same type that the pretrained model
        gpu_available = False       #indicates if gpu should be used for predictions
    ): 
        """Create a AutoModelForMaskedLM from a pretrained model."""
        model = AutoModelForMaskedLM.from_pretrained(name_or_path)
        if(gpu_available):
            print("------------------Loading Model into GPU------------------")
            model = AutoModelForMaskedLM.from_pretrained(name_or_path).to('cuda:3')
        return Predictor(tokenizer, model, gpu_available)


## Testing

In [4]:
import torch

print(torch.cuda.is_available())

print(torch.cuda.device_count())

!nvidia-smi

True
1
Thu Dec  1 15:01:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:61:00.0 Off |                    0 |
| N/A   32C    P0    39W / 250W |      2MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+

In [5]:
from CodeCheckList.masker import Masker

checkpoint = "huggingface/CodeBERTa-small-v1"
python_language = "python"

bert_tokenizer = CodeTokenizer.from_pretrained(checkpoint, python_language)
predictor = Predictor.from_pretrained(checkpoint, bert_tokenizer)
code_masker = Masker(bert_tokenizer)

------------------Loading Model into GPU------------------


In [6]:
code = "def multiply_numbers(a,b):\n    return a*b"
target_node_type = "*"

masked_code_encoding = code_masker(code, bert_tokenizer(code), bert_tokenizer.node_types.index(target_node_type))
masked_code = bert_tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == bert_tokenizer.tokenizer.bos_token_id or 
            token_id == bert_tokenizer.tokenizer.eos_token_id else True, masked_code_encoding['input_ids'])))
print(masked_code)

def multiply_numbers(a,b):
    return a<mask>b


In [7]:
predictions = predictor(masked_code_encoding, bert_tokenizer.tokenizer(code, return_tensors='pt'), 5)

print(predictions)

['def multiply_numbers(a,b):\n    return a,b', 'def multiply_numbers(a,b):\n    return a*b', 'def multiply_numbers(a,b):\n    return a+b', 'def multiply_numbers(a,b):\n    return a**b', 'def multiply_numbers(a,b):\n    return a/b']
